In [36]:
CODE_DIR = '/home/shreyas/Documents/git/wheelai/'
#DATA_DIR = '/media/shreyas/OS/ML_DATA/wheelai/kerakart/'
DATA_DIR = '/media/shreyas/OS/ML_DATA/wheelai/kerakart/sample/'
traindata_path = DATA_DIR + 'train/'
validdata_path = '/media/shreyas/OS/ML_DATA/wheelai/kerakart/valid/'
results_path = '/media/shreyas/OS/ML_DATA/wheelai/kerakart/results/'

In [37]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16

%matplotlib inline



### Finetune and Train

In [38]:
batch_size = 32
epochs = 3

In [39]:
def get_batches(path, class_mode='categorical', gen=image.ImageDataGenerator(rescale=1./255), \
                shuffle=True, target_size=(224,224), batch_size=1):
    return gen.flow_from_directory(path, class_mode=class_mode, batch_size=batch_size, \
                                   target_size=target_size, shuffle=shuffle)

def get_steps(batches, batch_size):
    steps = int(batches.samples/batch_size)
    return (steps if batches.samples%batch_size==0 else (steps+1))


In [40]:
train_b = get_batches(traindata_path, batch_size=batch_size)
valid_b = get_batches(validdata_path, batch_size=batch_size)

Found 24000 images belonging to 3 classes.
Found 4254 images belonging to 3 classes.


In [41]:
train_steps = get_steps(train_b, batch_size)
valid_steps = get_steps(valid_b, batch_size)
num_class = train_b.num_class

In [42]:
train_labels = train_b.classes
valid_labels = valid_b.classes

In [43]:
y_train = keras.utils.to_categorical(train_labels)
y_valid = keras.utils.to_categorical(valid_labels)

In [44]:
print(train_labels.shape, valid_labels.shape, y_train.shape, y_valid.shape)

(24000,) (4254,) (24000, 3) (4254, 3)


In [45]:
trn_b = get_batches(traindata_path, class_mode=None, shuffle=False, batch_size=batch_size)
val_b = get_batches(validdata_path, class_mode=None, shuffle=False, batch_size=batch_size)

Found 24000 images belonging to 3 classes.
Found 4254 images belonging to 3 classes.


In [46]:
gen_augdata = image.ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, 
                                      shear_range=0.2, zoom_range=0.2)
augtrain_b = get_batches(traindata_path, gen=gen_augdata, class_mode='categorical',\
                         batch_size=batch_size)

Found 24000 images belonging to 3 classes.


In [47]:
def vgg_conv():
    model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Conv2D][-1]
    conv_layers = layers[:15]
    conv_model = Sequential(conv_layers)
    return conv_model


def top_layer(input_shape):
    model = Sequential()
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Dropout(0.1))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(256, activation='elu'))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='elu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_class, activation='softmax'))
    
    return model 

In [48]:
def basic_model(input_shape):
    model = Sequential()
    #model.add(BatchNormalization(axis=1, input_shape=input_shape))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu',input_shape=input_shape))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(Flatten())    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(3, activation='softmax'))
    return model

In [49]:
def get_ft(trn_b, train_steps, val_b, valid_steps):
    model = vgg_conv()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    trn_ft = model.predict_generator(trn_b, train_steps)
    val_ft = model.predict_generator(val_b, valid_steps)
    print(trn_ft.shape, val_ft.shape)

    np.save(results_path + 'trn_ft.npy', trn_ft)
    np.save(results_path + 'val_ft.npy', val_ft)

In [ ]:
get_ft(trn_b, train_steps, val_b, valid_steps)

In [ ]:
X_train = np.load(results_path + 'trn_ft.npy')
X_valid = np.load(results_path + 'val_ft.npy')

In [ ]:
model = top_layer(X_train.shape[1:])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=2e-5), metrics=['accuracy'])

In [ ]:
#vgg_model.fit(X_train, y_train, epochs=2, batch_size=1, verbose=1, \
#          validation_data=(X_valid, y_valid))
vgg_model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=10,\
                   validation_data=valid_b, validation_steps=valid_steps)

In [ ]:
#model.save_weights(results_path+'trial_bottleneck.h5')

In [ ]:
filepath1 = results_path+'vgg_bottleneck.h5'
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=True,\
                             save_weights_only=True, mode='min', period=1)
callbacks1=[checkpoint1]

In [ ]:
model.fit(X_train, y_train, epochs=2, batch_size=1, verbose=1, \
          callbacks=callbacks1, validation_data=(X_valid, y_valid))

In [50]:
def vgg_model():
    base_model = vgg_conv()
    for layer in base_model.layers[:11]: layer.trainable=False
    print (base_model.output_shape[1:])
    top_model = top_layer(base_model.output_shape[1:])
    top_model.load_weights(results_path+'vgg_bottleneck.h5')
    
    base_model.add(top_model)
    
    return base_model

In [51]:
vgg_model = vgg_model()
vgg_model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=2e-5, momentum=0.9), metrics=['accuracy'])

(14, 14, 512)


In [52]:
filepath2 = results_path+'vgg_ft.h5'
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1,\
                              save_best_only=True, mode='min', period=1)
callbacks2=[checkpoint2]

In [56]:
vgg_model.fit_generator(train_b, steps_per_epoch=train_steps, epochs=21, callbacks=callbacks2,\
                        validation_data=valid_b, validation_steps=valid_steps)

Epoch 1/21
750/750 [==============================] - 230s - loss: 0.4461 - acc: 0.8115 - val_loss: 0.3138 - val_acc: 0.8653
Epoch 2/21
750/750 [==============================] - 229s - loss: 0.2755 - acc: 0.8831 - val_loss: 0.2798 - val_acc: 0.8775
Epoch 3/21
 14/750 [..............................] - ETA: 205s - loss: 0.2227 - acc: 0.9062

KeyboardInterrupt: 

In [57]:
filepath3 = results_path+'vgg_augmented_ft.h5'
checkpoint3 = ModelCheckpoint(filepath3, monitor='val_loss', verbose=1,\
                              save_best_only=True, mode='min', period=1)
callbacks3=[checkpoint3]

In [59]:
vgg_model.fit_generator(augtrain_b, steps_per_epoch=train_steps, epochs=13, callbacks=callbacks3,\
                        validation_data=valid_b, validation_steps=valid_steps)

Epoch 1/13
750/750 [==============================] - 228s - loss: 0.5957 - acc: 0.7411 - val_loss: 0.6193 - val_acc: 0.8505
Epoch 2/13
750/750 [==============================] - 232s - loss: 0.5145 - acc: 0.7604 - val_loss: 0.6579 - val_acc: 0.8434
Epoch 3/13
750/750 [==============================] - 226s - loss: 0.5050 - acc: 0.7598 - val_loss: 0.6999 - val_acc: 0.8235
Epoch 4/13
750/750 [==============================] - 226s - loss: 0.4971 - acc: 0.7677 - val_loss: 0.7045 - val_acc: 0.8068
Epoch 5/13
750/750 [==============================] - 226s - loss: 0.4733 - acc: 0.7789 - val_loss: 0.7160 - val_acc: 0.8129
Epoch 6/13
750/750 [==============================] - 226s - loss: 0.4754 - acc: 0.7758 - val_loss: 0.7020 - val_acc: 0.8202
Epoch 7/13
750/750 [==============================] - 226s - loss: 0.4709 - acc: 0.7770 - val_loss: 0.7030 - val_acc: 0.8096
Epoch 8/13
750/750 [==============================] - 227s - loss: 0.4545 - acc: 0.7827 - val_loss: 0.7225 - val_acc: 0.7967


In [60]:
model.save_weights(results_path+'vgg_ft_augmenteddata.h5')

In [ ]:
preds.shape

In [ ]:
print (preds[:5])
img = batches.filenames
print (img[:5])

In [ ]:
from PIL import Image
Image.open(DATA_DIR+'test/'+img[2])

In [ ]:
save_array(results_path + 'test_predictions.dat', preds)
save_array(results_path + 'imagefiles.dat', img)

### Validate Predictions
Lets plot -
1. A few correct labels at random
2. A few incorrect labels at random
3. Most confident correct predictions of each class
4. Most confident incorrect predictions of each class
5. Most uncertain labels (probabilites close to 0.5)

In [ ]:
vgg.model.load_weights(filepath)

In [ ]:
val_batches, probs = vgg.test(validdata_path, batch_size=batch_size*2)

In [ ]:
img = val_batches.filenames
expected_labels = val_batches.classes

our_predictions = probs[:,0]
other_predictions = np.round(probs[:,1])
our_labels = np.round(1-our_predictions)

In [ ]:
from keras.preprocessing import image

def plots_idx(idx, titles=None):
    plots([image.load_img(validdata_path + img[i]) for i in idx], titles=titles)

In [ ]:
n_view = 4

In [ ]:
correct = np.where(our_labels==expected_labels)[0]
print ("Found %d correct labels" % len(correct))
idx = permutation(correct)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:
incorrect = np.where(our_labels!=expected_labels)[0]
print ("Found %d incorrect labels" % len(incorrect))
idx = permutation(incorrect)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:

#3a. The images we most confident were cats, and are actually cats
correct_cats = np.where((our_labels==0) & (our_labels==expected_labels))[0]
print ("Found %d confident correct cats labels" % len(correct_cats))
most_correct_cats = np.argsort(our_predictions[correct_cats])[::-1][:n_view]
plots_idx(correct_cats[most_correct_cats], our_predictions[correct_cats][most_correct_cats])

In [ ]:
#3b. The images we most confident were dogs, and are actually dogs
correct_dogs = np.where((our_labels==1) & (our_labels==expected_labels))[0]
print ("Found %d confident correct dogs labels" % len(correct_dogs))
most_correct_dogs = np.argsort(our_predictions[correct_dogs])[:n_view]
plots_idx(correct_dogs[most_correct_dogs], our_predictions[correct_dogs][most_correct_dogs])

In [ ]:
#4a. The images we were most confident were cats, but are actually dogs
incorrect_cats = np.where((our_labels==0) & (our_labels!=expected_labels))[0]
print ("Found %d incorrect cats" % len(incorrect_cats))
if len(incorrect_cats):
    most_incorrect_cats = np.argsort(our_predictions[incorrect_cats])[::-1][:n_view]
    plots_idx(incorrect_cats[most_incorrect_cats], our_predictions[incorrect_cats][most_incorrect_cats])

In [ ]:
#4b. The images we were most confident were dogs, but are actually cats
incorrect_dogs = np.where((our_labels==1) & (our_labels!=expected_labels))[0]
print ("Found %d incorrect dogs" % len(incorrect_dogs))
if len(incorrect_dogs):
    most_incorrect_dogs = np.argsort(our_predictions[incorrect_dogs])[:n_view]
    plots_idx(incorrect_dogs[most_incorrect_dogs], our_predictions[incorrect_dogs][most_incorrect_dogs])

In [ ]:
#5. The most uncertain labels (ie those with probability closest to 0.5).
most_uncertain = np.argsort(np.abs(our_predictions-0.5))
plots_idx(most_uncertain[:n_view], our_predictions[most_uncertain])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)
plot_confusion_matrix(cm, val_batches.class_indices)

In [ ]:
preds = load_array(results_path + 'test_predictions.dat')
filenames = load_array(results_path + 'imagefiles.dat')

In [ ]:
isdog = preds[:,1]
print ("Raw Predictions: "+ str(isdog[:5]))
print ("Mid Predictions: "+str(isdog[(isdog<.6)&(isdog>0.4)]))
print ("Edge Predictions: "+str(isdog[(isdog<0.02)&(isdog>.98)]))

In [ ]:
np.amax(isdog)

In [ ]:
#Extract imageIds from the filenames in our test/unknown directory 
filenames = batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

In [ ]:
subm = np.stack([ids,isdog], axis=1)
subm[:5]

In [ ]:
%cd $DATA_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')